In [1]:
import pandas as pd
import numpy as np

In [141]:
def weighted_median1(values, weights):
    sorted_indices = np.argsort(values)
    sorted_values = sorted(values)
    sorted_weights = sorted(weights)

    cumulative_weights = np.cumsum(sorted_weights)
    total_weight = np.sum(sorted_weights)

    # Find the median index
    median_index = np.searchsorted(cumulative_weights, total_weight / 2.0)

    return sorted_values[median_index]

In [133]:
#def weighted_median(df, val, weight):
#    df_sorted = df.sort_values(val)
#    cumsum = df_sorted[weight].cumsum()
#    cutoff = df_sorted[weight].sum() / 2.
#    return df_sorted[cumsum >= cutoff][val].iloc[0]

#weighted_median(df, 'impwealth', 'indweight')

In [2]:
dataset = pd.read_csv('data/weather_data_set.txt', sep="\t",  header=None)
groundtruth = pd.read_csv('data/weather_ground_truth.txt', sep="\t")
iti = 3

In [ ]:
dataset

Main CRH Method

In [113]:
dataset.sort_values(by=[0], inplace=True)

# Calculate statistics about the data
nof = len(dataset)
# Get the entry list
list_entry = dataset[0].unique()
# Get the source list
list_source = dataset[2].unique()
# Number of sources
nos = len(list_source)
# Number of entries
noe = len(list_entry)

# Modify entry_ia so that the following works
entry_ia = np.append(np.arange(0, noe), nof)

# Initialization
ini_truth = []
weight = np.ones(nos) / nos

weight_vector = weight[dataset[2].values-1]
weight_matrix = pd.DataFrame(weight_vector, columns=['weight'],index = dataset.index)
# Calculate initial truth entry by entry

stand_error=[]

In [116]:
weight_matrix.loc[ [333, 153]]

,weight
333,0.111111
153,0.111111


In [110]:
weight_matrix.index

Index([  333,   153,   334,   154,   335,   155,   336,   675,   855,   156,
       ...
       15650, 15657, 15658, 15659, 15666, 15667, 15668, 15675, 15676, 15677],
      dtype='int64', length=16038)

In [122]:
for entry_id in list_entry:
    val = dataset[dataset[0] == entry_id].values[0,1]
 
    if not val.lstrip('-+').isdigit():  # Categorical data
        same_entry = dataset[dataset[0]==entry_id]
        tempvalue = same_entry[1].values
        # Get unique value list on the same entry
        temp_list = same_entry[1].unique()
        wv = []

        # Calculate the weighted occurrence for this entry
        for k in range(len(temp_list)):
            wv.append(np.sum(((tempvalue == temp_list[k]) * weight_matrix.loc[same_entry.index].values)))

        # Get the voting result
        I = np.argmax(wv)
        ini_truth.append([entry_id, temp_list[I]])

    else:  # Numerical data
        
        same_entry = dataset[dataset[0]==entry_id]
        
        tempvalue = same_entry[1].values
        tempvalue_int = [int(x) for x in tempvalue]
        median = np.median(tempvalue_int)
        
        # Calculate median for this entry
        ini_truth.append([entry_id, median])
        stand_error.append([entry_id,np.std(tempvalue_int) ])
index_truth = [] #ini_truth.copy()
truth_matrix = pd.DataFrame(ini_truth, columns=['entry_id', 'truth'])
std_matrix = pd.DataFrame(stand_error, columns=['entry_id', 'error'])

In [124]:
# Initialize other parameters
cat_count = np.zeros(nos)  # Count of categorical data for each source
con_count = np.zeros(nos)  # Count of continuous data for each source

In [142]:
for i in range(iti):
    score1 = np.zeros(nos)
    score2 = np.zeros(nos)

    # Update weight
    
    for j in range(len(dataset)):
        val = dataset.loc[j].values[1]
        entry_id = dataset.loc[j].values[0]
        source_id = dataset.loc[j].values[2]
        true_val=truth_matrix[truth_matrix['entry_id'] == entry_id].values[0,1]
        if not val.lstrip('-+').isdigit():
            true_val=truth_matrix[truth_matrix['entry_id'] == entry_id].values[0,1]
            score1[source_id-1] += int(true_val != val)  #we count non-matching TypeError: weighted_median() missing 1 required positional argument: 'weight'values
            #if i == 2:
            cat_count[source_id-1] += 1
        else:  # Numerical data
            error=std_matrix[std_matrix['entry_id'] == entry_id].values[0,1]
            if (error != 0 ):
                score2[source_id-1] += np.abs( int(val) - int(true_val)) / error

            #if i == 2:
            con_count[source_id-1] += 1
    score1 /= cat_count
    score2 /= con_count
    score1 /= sum(score1)
    score2 /= sum(score2)

    score = score1 + score2
    norm_score = np.max(score)
    w = score / norm_score
    weight = -np.log(w) + 0.00001

    weight_vector = weight[dataset[2].values-1]
    weight_matrix = pd.DataFrame(weight_vector, columns=['weight'],index = dataset.index)
    for entry_id in list_entry:
        val = dataset[dataset[0] == entry_id].values[0,1]
        if not val.lstrip('-+').isdigit():  # Categorical data
            same_entry = dataset[dataset[0]==entry_id]
            tempvalue = same_entry[1].values
            temp_list = same_entry[1].unique()
            wv = []
            for k in range(len(temp_list)):
                wv.append(np.sum(((tempvalue == temp_list[k]) * weight_matrix.loc[same_entry.index].values)))
            I = np.argmax(wv)
            index_truth.append([entry_id, temp_list[I]])
        else:  # Numerical data
            same_entry = dataset[dataset[0]==entry_id]
            tempvalue = same_entry[1].values
            tempvalue_int = [int(x) for x in tempvalue]
            
            tempweight=weight_matrix.loc[same_entry.index].values
            wm = weighted_median1(tempvalue_int, tempweight)
            index_truth.append([entry_id, wm])

    print(len(index_truth))
    truth_matrix = index_truth.loc[dataset[0].values].reset_index(drop=True)[2]





        
       
 

8408


AttributeError: 'list' object has no attribute 'loc'

In [ ]:
for i in range(iti):
        score1 = np.zeros(nos)
        score2 = np.zeros(nos)

        # Update weight
        
        for j in range(len(dataset)):
            print(dataset.loc[j])
            val = dataset.loc[j].values[1]
            if not val.isdigit():
           # if pd.api.types.is_string_dtype(dataset.at[j, 1]):  # Categorical data
                score1[dataset.at[j, 2]] += int(truth_matrix[j] != dataset.at[j, 1])

                if i == 2:
                    cat_count[dataset.at[j, 2]] += 1
            else:  # Numerical data
                score2[dataset.at[j, 2]] += np.abs((pd.to_numeric(dataset.at[j, 1]) - truth_matrix[j]) / std_matrix[j])

                if i == 2:
                    con_count[dataset.at[j, 2]] += 1

        score1 /= cat_count
        score2 /= con_count
        score1 /= np.sum(score1)
        score2 /= np.sum(score2)

        # Sum up the distance for categorical and continuous data
        score = score1 + score2

        # Calculate weight for sources
        norm_score = np.max(score)
        w = score / norm_score
        weight = -np.log(w) + 0.00001
        weight_matrix = weight[dataset[2].values]


    
        # Update truth
        for j in range(noe):
            if pd.api.types.is_string_dtype(dataset.at[entry_ia[j], 1]):  # Categorical data
                tempvalue = dataset.loc[entry_ia[j]:entry_ia[j + 1] - 1, 1]
                temp_list = tempvalue.unique()
                length_list = len(temp_list)

                wv = []
                for k in range(length_list):
                    wv.append(np.sum((tempvalue == temp_list[k]) * weight_matrix[entry_ia[j]:entry_ia[j + 1]]))

                I = np.argmax(wv)
                index_truth.at[j + 1, 2] = temp_list[I]

            else:  # Numerical data
                tempvalue = pd.to_numeric(dataset.loc[entry_ia[j]:entry_ia[j + 1] - 1, 1])
                tempweight = weight_matrix[entry_ia[j]:entry_ia[j + 1]]

                # Update truth by weighted median
                index_truth.at[j + 1, 2] = weighted_median(tempvalue.values, tempweight)

        truth_matrix = index_truth.loc[dataset[0].values].reset_index(drop=True)[2]


In [ ]:
nof = len(dataset)
list_entry, entry_ia, entry_ic = np.unique(dataset["entry_id"], return_index=True, return_inverse=True)
list_source, _, source_ic = np.unique(dataset["source_index"], return_index=True, return_inverse=True)
nos = len(list_source)
noe = len(entry_ia)
entry_ia = np.append(entry_ia, nof + 1)

In [ ]:
# Initialization
ini_truth = []
weight = np.ones(nos) / nos
weight_matrix = weight[source_ic]

In [ ]:
noe

In [ ]:
for i in range(noe):
    ini_truth.append([dataset.iloc[entry_ia[i], 0], None])
    if dataset.iloc[entry_ia[i], 1].replace('.', '', 1).isdigit():  # numerical data
        tempvalue = dataset.iloc[entry_ia[i]: entry_ia[i + 1] - 1, 1].astype(float)
        ini_truth[i][1] = np.median(tempvalue)
    else:  # categorical data
        tempvalue = dataset.iloc[entry_ia[i]: entry_ia[i + 1] - 1, 1]
        temp_list, counts = np.unique(tempvalue, return_counts=True)
        wv = np.zeros(len(temp_list))
        for k in range(len(temp_list)):
            wv[k] = sum((tempvalue == temp_list[k]) * weight_matrix[entry_ia[i]: entry_ia[i + 1] - 1])
        ini_truth[i][1] = temp_list[np.argmax(wv)]


In [ ]:
dataset = pd.DataFrame(dataset, columns=["entry_id", "observation", "source_index"])
dataset = dataset.sort_values(by="entry_id")

# Calculate statistics about the data
nof = len(dataset)
list_entry, entry_ia, entry_ic = np.unique(dataset["entry_id"], return_index=True, return_inverse=True)
list_source, _, source_ic = np.unique(dataset["source_index"], return_index=True, return_inverse=True)
nos = len(list_source)
noe = len(entry_ia)
entry_ia = np.append(entry_ia, nof + 1)

# Initialization
ini_truth = []
weight = np.ones(nos) / nos
weight_matrix = weight[source_ic]

for i in range(noe):
    ini_truth.append([dataset.iloc[entry_ia[i], 0], None])
    if dataset.iloc[entry_ia[i], 1].replace('.', '', 1).isdigit():  # numerical data
        tempvalue = dataset.iloc[entry_ia[i]: entry_ia[i + 1] - 1, 1].astype(float)
        ini_truth[i][1] = np.median(tempvalue)
    else:  # categorical data
        tempvalue = dataset.iloc[entry_ia[i]: entry_ia[i + 1] - 1, 1]
        temp_list, counts = np.unique(tempvalue, return_counts=True)
        wv = np.zeros(len(temp_list))
        for k in range(len(temp_list)):
            wv[k] = sum((tempvalue == temp_list[k]) * weight_matrix[entry_ia[i]: entry_ia[i + 1] - 1])
        ini_truth[i][1] = temp_list[np.argmax(wv)]

index_truth = ini_truth.copy()
truth_matrix = [ini_truth[i][1] for i in entry_ic]
std_matrix = np.zeros(len(entry_ic))

cat_count = np.zeros(nos)
con_count = np.zeros(nos)

i = 1
while i <= iti:
    i += 1
    score1 = np.zeros(nos)
    score2 = np.zeros(nos)

    for j in range(nof):
        if dataset.iloc[j, 1].replace('.', '', 1).isdigit():
            score2[dataset.iloc[j, 2]] += abs((float(dataset.iloc[j, 1]) - truth_matrix[j]) / std_matrix[j])
            if i == 2:
                con_count[dataset.iloc[j, 2]] += 1
        else:
            score1[dataset.iloc[j, 2]] += (truth_matrix[j] != dataset.iloc[j, 1])
            if i == 2:
                cat_count[dataset.iloc[j, 2]] += 1

    score1 /= cat_count
    score2 /= con_count
    score1 /= np.sum(score1)
    score2 /= np.sum(score2)

    score = score1 + score2
    norm_score = max(score)
    w = score / norm_score
    weight = -np.log(w) + 0.00001
    weight_matrix = weight[source_ic]

    for j in range(noe):
        if dataset.iloc[entry_ia[j], 1].replace('.', '', 1).isdigit():
            tempvalue = dataset.iloc[entry_ia[j]: entry_ia[j + 1] - 1, 1].astype(float)
            tempweight = weight_matrix[entry_ia[j]: entry_ia[j + 1] - 1]
            index_truth[j][1] = weightedMedian(tempvalue, tempweight)
        else:
            tempvalue = dataset.iloc[entry_ia[j]: entry_ia[j + 1] - 1, 1]
            temp_list, counts = np.unique(tempvalue, return_counts=True)
            wv = np.zeros(len(temp_list))
            for k in range(len(temp_list)):
                wv[k] = sum((tempvalue == temp_list[k]) * weight_matrix[entry_ia[j]: entry_ia[j + 1] - 1])
            index_truth[j][1] = temp_list[np.argmax(wv)]

    truth_matrix = [index_truth[i][1] for i in entry_ic]

weight = np.column_stack((list_source, weight))
return index_truth, weight, ini_truth

In [ ]:
import numpy as np
from statistics import median
import csv
# Convert 'importfile_data' and 'importfile_gt' functions as needed to read the data files.

def evaluate_weather(crh_truth, ground_truth):
    # Implement the evaluation of the CRH results against the ground truth in this function.
    # Return the error rate and MNAD (Mean Normalized Absolute Deviation) as required.
  pass




In [ ]:

def weighted_median(D, W):
    if D.shape != W.shape:
        raise ValueError('The dimensions of the input-matrices must match.')

    # normalize the weights, such that: sum ( w_ij ) = 1
    WSum = np.sum(W)
    W = W / WSum

    # transformation of the input-matrices to vectors
    d = D.flatten()
    w = W.flatten()

    # sort the vectors
    sorted_indices = np.argsort(d)
    d_sorted = d[sorted_indices]
    w_sorted = w[sorted_indices]

    # Calculate cumulative sums of the weights
    sumVec = np.cumsum(w_sorted)

    wMed = None
    j = 0

    while wMed is None:
        if sumVec[j] >= 0.5:
            wMed = d_sorted[j]
        j += 1

    if (np.sum(w_sorted[:j - 1]) > 0.5) and (np.sum(w_sorted[j:]) > 0.5):
        raise ValueError('The weighted median could not be calculated.')

    return wMed

def CRH_weather(dataset, iti):
    if not np.all(np.diff(dataset[:, 0]) >= 0):
        dataset = dataset[dataset[:, 0].argsort()]

    nof = np.array(dataset).shape[0]
    list_entry, entry_ia, entry_ic = np.unique(dataset[:, 0], return_index=True, return_inverse=True)
    list_source, _, source_ic = np.unique(dataset[:, 2], return_index=True, return_inverse=True)
    nos = len(list_source)
    noe = len(entry_ia)

    entry_ia = np.append(entry_ia, nof + 1)

    ini_truth = []
    weight = np.ones(nos) / nos
    weight_matrix = weight[source_ic]
    standerror = np.zeros(nof)

    for i in range(noe):
        ini_truth.append([dataset[entry_ia[i], 0], None])

        if not dataset[entry_ia[i], 1].replace('.', '', 1).isdigit():
            tempvalue = dataset[entry_ia[i]: entry_ia[i + 1], 1]
            temp_list, counts = np.unique(tempvalue, return_counts=True)
            wv = [np.sum((tempvalue == val) * weight_matrix[entry_ia[i]: entry_ia[i + 1]]) for val in temp_list]
            I = np.argmax(wv)
            ini_truth[i][1] = temp_list[I]
        else:
            tempvalue = dataset[entry_ia[i]: entry_ia[i + 1], 1].astype(float)
            ini_truth[i][1] = median(tempvalue)
            standerror[i] = np.std(tempvalue) + 0.1

    index_truth = ini_truth
    truth_matrix = np.array([x[1] for x in ini_truth])[entry_ic]
    std_matrix = standerror[entry_ic]

    cat_count = np.zeros(nos)
    con_count = np.zeros(nos)

    i = 1
    while i <= iti:
        i += 1
        score1 = np.zeros(nos)
        score2 = np.zeros(nos)

        for j in range(nof):
            if not dataset[j, 1].replace('.', '', 1).isdigit():
                score1[int(dataset[j, 2])] += truth_matrix[j] != dataset[j, 1]
                if i == 2:
                    cat_count[int(dataset[j, 2])] += 1
            else:
                score2[int(dataset[j, 2])] += abs((float(dataset[j, 1]) - truth_matrix[j]) / std_matrix[j])
                if i == 2:
                    con_count[int(dataset[j, 2])] += 1

        score1 /= cat_count
        score2 /= con_count
        score1 /= np.sum(score1)
        score2 /= np.sum(score2)

        score = score1 + score2
        norm_score = np.max(score)
        w = score / norm_score
        weight = -np.log(w) + 0.00001
        weight_matrix = weight[source_ic]

        for j in range(noe):
            if not dataset[entry_ia[j], 1].replace('.', '', 1).isdigit():
                tempvalue = dataset[entry_ia[j]: entry_ia[j + 1], 1]
                temp_list, _ = np.unique(tempvalue, return_counts=True)
                wv = [np.sum((tempvalue == val) * weight_matrix[entry_ia[j]: entry_ia[j + 1]]) for val in temp_list]
                I = np.argmax(wv)
                index_truth[j][1] = temp_list[I]
            else:
                tempvalue = dataset[entry_ia[j]: entry_ia[j + 1], 1].astype(float)
                tempweight = weight_matrix[entry_ia[j]: entry_ia[j + 1]]
                index_truth[j][1] = weighted_median(tempvalue, tempweight)

        truth_matrix = np.array([x[1] for x in index_truth])[entry_ic]

    weight = np.column_stack((list_source, weight))
    return index_truth, weight, ini_truth

# Example usage:
# Assuming 'dataset' is a 2D NumPy array.
# iti = 10  # Maximum number of iterations
# index_truth, weight, ini_truth = CRH_weather(dataset, iti)


In [ ]:
# Import data and ground truth
weather =  importfile_data('data/weather_data_set.txt')
ground_truth = importfile_gt('data/weather_ground_truth.txt')

#weather[:, 0]=weather[:, 0].astype(float).astype(int)
# Set minimum and maximum iterations
min_iti = 3
max_iti = 3
result_CRH = np.zeros((max_iti - min_iti + 1, 3))

i = 0
# Start CRH iteration
for iti in range(min_iti, max_iti + 1):
  i += 1
  print(f"{i}th run: iteration = {iti} ...")

  # Run CRH
  crh_truth, crh_weight, voting_median_truth = CRH_weather(weather, iti)

  # Evaluate CRH result for current iteration
  error_rate, MNAD = evaluate_weather(crh_truth, ground_truth)
  result_CRH[i - 1, :] = [iti, error_rate, MNAD]

# Save CRH result
print("saving CRH result...")
np.savetxt('result_CRH.txt', result_CRH, fmt='%1.4f', delimiter='\t')

# Voting/Median method
# If you want to evaluate the results from the Voting/Median method, you can
# uncomment the following code and implement the necessary evaluation functions.

# Evaluate Voting/Median result for current iteration
# error_rate, MNAD = evaluate_truthmine(crh_truth, ground_truth)
# result_vm = [error_rate, MNAD]

# Save Voting/Median result
# print("saving Voting/Median result...")
# np.savetxt('result_vm.txt', result_vm, fmt='%1.4f', delimiter='\t')

In [ ]:
weather[:, 0]=weather[:, 0].astype(float).astype(int)

In [ ]:
weather

In [ ]:
weather[:, 0].astype(np.float)
nums_str_np = np.asarray(weather[:, 0])
nums_int_np = nums_str_np.astype('int')
nums_int_np

In [ ]:
x = np.array([[1, 2, 4, 7, 0],[0,0,0,0,0]])

np.diff(x[:,0])